# Практическая работа №2: Исследование алгоритмов формирования аддитивных цепочек

Выполнил студент гр. 1304 Стародубов Максим. Вариант №24.

## Цель работы

Формирование представления о аддитивных цепочках, выработать умение составлять и применять алгоритмы для нахождения минимальных аддитивных цепочек для заданного числа, привить навык использования систем компьютерной математики для реализации алгоритмов.

## Основные теоретические положения

### Алгоритм Брауэра

В общем виде алгоритм можно представить следующим образом:<br>
$
\begin{equation*}
B_k(n) = 
 \begin{cases}
   1, 2, 3, ..., 2^k-1 &\text{, если $n < 2^k$}\\
   B_k(q), 2q, 4q, 8q, ..., 2^kq, n &\text{, если $n \geq 2^k, q = \lfloor {{n} \over {2^k}} \rfloor$}
 \end{cases}
\end{equation*}
$

Вычисления происходят следующим образом:<br>
$d = 2^k$<br>
$q_1 = \lfloor {n \over d} \rfloor$<br>
$r_1 = n \text{ mod } d \Rightarrow n = q_1d + r_1$

$q_2 = \lfloor {q_1 \over d} \rfloor$<br>
$r_2 = q_1 \text{ mod } d \Rightarrow q_1 = q_2d + r_2$

...

$q_s = \lfloor {q_{s-1} \over d} \rfloor$<br>
$r_s = q_{s-1} \text{ mod } d \Rightarrow q_s = q_{s-1}d + r_s$

Данные вычисления оканчиваются как только $q_s < d$.

Используя найденные $q_i$ можно построить разложение:<br>
$n = 2^kq_1+r_1=2^k(2^kq_2+r_2)+r_1 = ... = 2^k(2^k(...(2^kq_s+r_s)...)+r_2)+r_1$

### Алгоритм Яо

Введем следующие обозначения:<br>
$k \geq 2$<br>
$n = \sum_{i=0}^ia_i2^{ik}$

$d(z) = \sum_{i: a_i = z}2^{ik}$

Построение базовой последовательности:<br>
$1, 2, 4, 8, ..., 2^{\lambda(n)}$

Для всех $z \in \{1, 2, ...,2^k-1\}$ происходит вычисление $d(z)$.

Для всех $z \in \{1, 2, ...,2^k-1\}$ происходит вычисление $zd(z)$

Получено разложение $n$:<br>
$n = \sum_{z=1}^{2^k-1}zd(z)$

### Алгоритм дробления вектора индексов

* 1. Внешний цикл производит перебор возможных $m$ - длин векторов индексов. Поисходит выбор числа $q \in N, 0 \leq q \leq m$.

* 2. Внутренний цикл производит перебор всех фиксированных чистей векторов индексов $\{r_1, ...,r_q\}$. Для каждой фиксированной части происходит вычисление $a_{min} = a_{q+1} + m - q$ и $a_{max} = a^{q+1} \cdot 2^{m-q}$.
    
    * 2.1. Если $n \notin [a_{min}, a_{max}]$, то происходит переход к следующей фиксированной части $\{r_1, ...,r_q\}$.
    
    * 2.2. Если $n \in [a_{min}, a_{max}]$, то организуется внутренний цикл перебора меняющейся части $\{\rho_{q+1}, ..., \rho_{m}\}$.
        
        * 2.2.1 Если обнаруживается $a_m = n$ - решение найдено.
    
        * 2.2.2 Если в цикле таких векторов не оказалось, то происходит переход к следующей фиксированной части.

* 3. Увеличиваем длину цепочки.

### Гипотеза Шольца-Брауэра

$ l(2^n - 1) \leq l(n) + n - 1 $

Данная гипотеза доказана для $ l^*(2^n - 1) \leq l^*(n) + n - 1 $.

## Постановка задачи

Реализовать точные и приближённые алгоритмы нахождения минимальных аддитивных цепочек с использованием системы компьютерной математики SageMath, провести анализ алгоритмов. Полученные результаты содержательно проинтерпретировать.

## Выполнение работы

### 1. Алгоритм Яо.
В блоке кода, расположенном ниже, реализован алгорим Яо.

In [1]:
'''
    this function calculates an approximate additive chain using the Yao algorithm
'''
def yao_algorithm(n, k):
    lambda_n = len(n.binary()) - 1
    
    # building a base chain
    chain = [1]
    for _ in range(lambda_n):
        chain.append(chain[-1] + chain[-1])

    # calculation of d(z) for all z != 0
    d = {}
    for i, a in enumerate(n.digits(2**k)):
        if a == 0:
            continue
        
        if a not in d:
            d[a] = chain[i * k]
            continue

        d[a] += chain[i * k]
        chain.append(d[a])
    
    # calculating z*d(z) using the SX binary method
    for a in d:
        x = d[a]            
        operations = a.binary()[1:]
        operations = operations.replace('1', 'SX').replace('0', 'S')
        for item in operations:
            if item == 'S':
                x += x
            else:
                x += d[a]
            
            # already calculated values are not added to the chain
            if x in chain:
                continue
            chain.append(x)
        d[a] = x
    
    item = d.popitem()
    
    # calculating the sum of all z*d(z)
    s = item[1]
    for a in d:
        s += d[a]
        chain.append(s)
    
    return chain

В блоках кода, расположенных ниже, произведены вычисления аддитивных цепочек для нескольких чисел с вариацией параметра $k$.

In [2]:
def print_chain_info(n, k):
    chain = yao_algorithm(n, k)
    print('n = {}, k = {}, chain length = {}\nchain = {}'.format(n, k, len(chain), chain))

In [3]:
print_chain_info(34, 2)

n = 34, k = 2, chain length = 8
chain = [1, 2, 4, 8, 16, 32, 17, 34]


In [4]:
print_chain_info(34, 3)

n = 34, k = 3, chain length = 7
chain = [1, 2, 4, 8, 16, 32, 34]


In [5]:
print_chain_info(180, 2)

n = 180, k = 2, chain length = 11
chain = [1, 2, 4, 8, 16, 32, 64, 128, 48, 132, 180]


In [6]:
print_chain_info(180, 3)

n = 180, k = 3, chain length = 12
chain = [1, 2, 4, 8, 16, 32, 64, 128, 24, 48, 132, 180]


In [7]:
print_chain_info(180, 4)

n = 180, k = 4, chain length = 12
chain = [1, 2, 4, 8, 16, 32, 64, 128, 80, 160, 176, 180]


In [8]:
print_chain_info(180, 5)

n = 180, k = 5, chain length = 13
chain = [1, 2, 4, 8, 16, 32, 64, 128, 5, 10, 20, 160, 180]


In [9]:
print_chain_info(180, 6)

n = 180, k = 6, chain length = 15
chain = [1, 2, 4, 8, 16, 32, 64, 128, 3, 6, 12, 13, 26, 52, 180]


In [10]:
print_chain_info(180, 7)

n = 180, k = 7, chain length = 15
chain = [1, 2, 4, 8, 16, 32, 64, 128, 3, 6, 12, 13, 26, 52, 180]


In [11]:
print_chain_info(180, 8)

n = 180, k = 8, chain length = 16
chain = [1, 2, 4, 8, 16, 32, 64, 128, 5, 10, 11, 22, 44, 45, 90, 180]


In [12]:
print_chain_info(1088, 3)

n = 1088, k = 3, chain length = 12
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1088]


In [13]:
print_chain_info(1088, 4)

n = 1088, k = 4, chain length = 14
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 272, 544, 1088]


In [14]:
print_chain_info(1088, 5)

n = 1088, k = 5, chain length = 12
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1088]


Полученные результаты сведены в таблицу, расположенную ниже.


| n | k | Аддитивная цепочка, полученная с помощью алгоритма Яо | Длина полученной<br>аддитивной цепочки | Длина минимальной<br>аддитивной цепочки |
|---|---|:--|---|---|
| 34 | 2 | \[1, 2, 4, 8, 16, 32, 17, 34\] | 8 | 7 |
| 34 | 3 | \[1, 2, 4, 8, 16, 32, 34\] | 7 | 7 |
| 180 | 2 | \[1, 2, 4, 8, 16, 32, 64, 128, 48, 132, 180\] | 11 | 10 |
| 180 | 3 | \[1, 2, 4, 8, 16, 32, 64, 128, 24, 48, 132, 180\] | 12 | 10 |
| 180 | 4 | \[1, 2, 4, 8, 16, 32, 64, 128, 80, 160, 176, 180\] | 12 | 10 |
| 180 | 5 | \[1, 2, 4, 8, 16, 32, 64, 128, 5, 10, 20, 160, 180\] | 13 | 10 |
| 180 | 6 | \[1, 2, 4, 8, 16, 32, 64, 128, 3, 6, 12, 13, 26, 52, 180\] | 15 | 10 |
| 180 | 7 | \[1, 2, 4, 8, 16, 32, 64, 128, 3, 6, 12, 13, 26, 52, 180\] | 15 | 10 |
| 180 | 8 | \[1, 2, 4, 8, 16, 32, 64, 128, 5, 10, 11, 22, 44, 45, 90, 180\] | 16 | 10 |
| 1088 | 3 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1088\] | 12 | 12 |
| 1088 | 4 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 272, 544, 1088\] | 14 | 12 |
| 1088 | 5 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1088\] | 12 | 12 |


Исходя из полученных результатов можно сделать вывод, что длина аддитивной цепочки, получаемой с помощью алгоритма Яо, зависит от переданного параметра $k$. В зависимости от параметра, длина аддитивной цепочки может как увеличиваться, так и уменьшаться. Алгоритм Яо не всегда дает оптимальное решение, причем существуют $n$, такие ,что алгоритм Яо не дает оптимального решения ни при каком заданном значении параметра $k$.

### 2. Алгоритм дробление вектора индексов.

В блоке кода, расположенном ниже, реализован алгоритм дробления вектора индексов.

In [15]:
'''
    this class produces sequential generation of index vectors of star chains
'''
class IndexVectorIterator:
    def __init__(self, length, start_degree=0):
        self._start_degree = start_degree
        self._current_vector = [start_degree + x for x in range(length)]
        self._is_start = True

    def __iter__(self):
        return self
    
    def __next__(self):
        if self._is_start:
            self._is_start = False
            return self._current_vector
        
        if all(x == 0 for x in self._current_vector):
            raise StopIteration
        
        for i in range(len(self._current_vector)-1, -1, -1):
            if self._current_vector[i] == 0:
                self._current_vector[i] = self._start_degree + i
                continue
            self._current_vector[i] -= 1
            break
        
        return self._current_vector

'''
    this function builds addition chain according to its index vector
'''
def get_responding_chain(index_vector, start=[1]):
    chain = copy(start)
    
    for i in index_vector:
        chain.append(chain[-1] + chain[i])
    
    return chain

'''
    this function finds the size of the changing part of the index vector 
    so that the number of permutations in it will be no more than 1000
'''
def get_variable_part_length(vector_length):
    variable_part_length = 0
    overshoot_rate = 1
    
    while overshoot_rate < 1000 and vector_length > 0:
        overshoot_rate *= vector_length
        vector_length -= 1
        variable_part_length += 1
    
    return variable_part_length

'''
    this function implements the index vector splitting algorithm
'''
def index_vector_splitting(n):
    if n == 1:
        return [1]
    
    if n == 2:
        return [1, 2]
    
    vector_length = len(bin(n)) - 3

    # external loop on lengths of vectors
    while True:
        
        # choice of lengths of constant and variable parts for a certain vector length
        constant_part_length = vector_length - get_variable_part_length(vector_length)
        # enumeration of all constant parts
        for constant_part in IndexVectorIterator(constant_part_length):
            
            constant_chain = get_responding_chain(constant_part)
            
            # the limits of the values that can be obtained by variation of the variable part
            a_min = constant_chain[-1] + vector_length - len(constant_part)
            a_max = constant_chain[-1] * 2^(vector_length - len(constant_part))
            
            # skipping enumeration of variable parts, if at a given constant part 
            # it is impossible to get the target value
            if not (a_min <= n <= a_max):
                continue
            
            # enumeration of all variable parts
            for variable_part in IndexVectorIterator(vector_length - len(constant_part), len(constant_part)):
                responding_chain = get_responding_chain(variable_part, constant_chain)
                if responding_chain[-1] == n:
                    return responding_chain
        
        vector_length += 1

В блоках, кода, расположенных ниже, приведены вычисления аддитивных цепочек для 5 значений $n > 1000$ с помощью алгоритма дробления вектора индексов.

In [16]:
from time import time
def print_index_vector_splitting_info(n):
    start_time = time()
    chain = index_vector_splitting(n)
    print('n = {}, chain length = {}, time = {}\nchain = {}'.format(n, len(chain), time() - start_time, chain))

In [17]:
print_index_vector_splitting_info(1088)

n = 1088, chain length = 12, time = 0.004200458526611328
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1088]


In [18]:
print_index_vector_splitting_info(1280)

n = 1280, chain length = 12, time = 0.003822803497314453
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1280]


In [19]:
print_index_vector_splitting_info(1160)

n = 1160, chain length = 13, time = 0.39328718185424805
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1152, 1160]


In [20]:
print_index_vector_splitting_info(1172)

n = 1172, chain length = 13, time = 0.5285634994506836
chain = [1, 2, 4, 8, 16, 32, 64, 128, 132, 260, 520, 1040, 1172]


In [21]:
print_index_vector_splitting_info(1920)

n = 1920, chain length = 13, time = 0.051650047302246094
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 640, 1280, 1920]


Полученные результаты сведены в таблицу, расположенную ниже.

| $n$ | Время<br>работы, с | Аддитивная цепочка, полученная с помощью алгоритма<br>дробления вектора индексов | Длина полученной<br>аддитивной цепочки | Длина минимальной<br>аддитивной цепочки |
|---|---|:--|---|---|
| 1088 | 0.003418445587158203 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1088\] | 12 | 12 |
| 1280 | 0.006251096725463867 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1280\] | 12 | 12 |
| 1160 | 0.4568898677825928 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1152, 1160\] | 13 | 13 |
| 1172 | 0.5387921333312988 | \[1, 2, 4, 8, 16, 32, 64, 128, 132, 260, 520, 1040, 1172\] | 13 | 13 |
| 1920 | 0.05087733268737793 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 640, 1280, 1920\] | 13 | 13 |

В блоке кода, расположенном ниже, произведены вычисления аддитивных цепочек для $n = 1920$ с помощью алгоритма Яо при различных значениях параметра $k$.

In [22]:
for k in range(2, 12):
    print_chain_info(1920, k)

n = 1920, k = 2, chain length = 14
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 768, 1152, 1920]
n = 1920, k = 3, chain length = 15
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 192, 384, 1536, 1920]
n = 1920, k = 4, chain length = 15
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 768, 1536, 1792, 1920]
n = 1920, k = 5, chain length = 17
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 96, 192, 224, 448, 896, 1920]
n = 1920, k = 6, chain length = 17
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 192, 384, 448, 896, 960, 1920]
n = 1920, k = 7, chain length = 16
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 384, 768, 896, 1792, 1920]
n = 1920, k = 8, chain length = 15
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 768, 1536, 1792, 1920]
n = 1920, k = 9, chain length = 21
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 3, 6, 12, 24, 48, 96, 192, 384, 1536, 1920]
n = 1920, k = 10, chain length = 22
chain = [1, 2, 4, 8, 16, 32, 64, 12

Исходя из полученных результатов можно сделать вывод, что существуют $n$, для которых алгоритм дробления вектора инедксов дает более оптимальное решение, чем алгоритм Яо (к примеру $n = 1920$). При этом время работы алгоритма дробления вектора индексов значительно выше, чем время работы алгритма Яо.

### 3. Гипотеза Шольца-Брауэра.

В блоке кода, расположенном ниже, реализована проверка гипотезы Шольца-Брауэра для всех натуральных $ 1 \leq n \leq 12 $ с использованием алгоритма дробления вектора индексов.

In [23]:
for n in range(1, 12 + 1):
    first_chain = index_vector_splitting(int(2^n - 1))
    second_chain = index_vector_splitting(n)
    print('| {} | {} | {} | {} | {} | {} |'.format(n, first_chain, len(first_chain), 
                                                   second_chain, len(second_chain), 
                                                   len(first_chain) <= len(second_chain) + n - 1))

| 1 | [1] | 1 | [1] | 1 | True |
| 2 | [1, 2, 3] | 3 | [1, 2] | 2 | True |
| 3 | [1, 2, 4, 6, 7] | 5 | [1, 2, 3] | 3 | True |
| 4 | [1, 2, 4, 5, 10, 15] | 6 | [1, 2, 4] | 3 | True |
| 5 | [1, 2, 4, 8, 10, 20, 30, 31] | 8 | [1, 2, 4, 5] | 4 | True |
| 6 | [1, 2, 4, 8, 16, 20, 21, 42, 63] | 9 | [1, 2, 4, 6] | 4 | True |
| 7 | [1, 2, 4, 8, 16, 32, 40, 42, 84, 126, 127] | 11 | [1, 2, 4, 6, 7] | 5 | True |
| 8 | [1, 2, 4, 8, 16, 17, 34, 68, 85, 170, 255] | 11 | [1, 2, 4, 8] | 4 | True |
| 9 | [1, 2, 4, 8, 16, 32, 64, 72, 73, 146, 292, 438, 511] | 13 | [1, 2, 4, 8, 9] | 5 | True |
| 10 | [1, 2, 4, 8, 16, 32, 64, 68, 136, 272, 340, 341, 682, 1023] | 14 | [1, 2, 4, 8, 10] | 5 | True |
| 11 | [1, 2, 4, 8, 16, 32, 64, 128, 256, 288, 292, 584, 585, 1170, 1755, 2047] | 16 | [1, 2, 4, 8, 10, 11] | 6 | True |
| 12 | [1, 2, 4, 8, 16, 32, 64, 128, 256, 272, 273, 546, 1092, 1365, 2730, 4095] | 16 | [1, 2, 4, 8, 12] | 5 | True |


Результаты вычислений сведены в таблицу, расположенную ниже.


| $n$ | Аддитивная цепочка для<br>$2^n - 1$ | Длина аддитивной цепочки | Аддитивная цепочка<br>для $n$ | Длина аддитивной цепочки | Проверка утверждения<br>гипотезы |
|---|---|---|---|---|---|
| 1 | [1] | 1 | [1] | 1 | True |
| 2 | [1, 2, 3] | 3 | [1, 2] | 2 | True |
| 3 | [1, 2, 4, 6, 7] | 5 | [1, 2, 3] | 3 | True |
| 4 | [1, 2, 4, 5, 10, 15] | 6 | [1, 2, 4] | 3 | True |
| 5 | [1, 2, 4, 8, 10, 20, 30, 31] | 8 | [1, 2, 4, 5] | 4 | True |
| 6 | [1, 2, 4, 8, 16, 20, 21, 42, 63] | 9 | [1, 2, 4, 6] | 4 | True |
| 7 | [1, 2, 4, 8, 16, 32, 40, 42, 84, 126, 127] | 11 | [1, 2, 4, 6, 7] | 5 | True |
| 8 | [1, 2, 4, 8, 16, 17, 34, 68, 85, 170, 255] | 11 | [1, 2, 4, 8] | 4 | True |
| 9 | [1, 2, 4, 8, 16, 32, 64, 72, 73, 146, 292, 438, 511] | 13 | [1, 2, 4, 8, 9] | 5 | True |
| 10 | [1, 2, 4, 8, 16, 32, 64, 68, 136, 272, 340, 341, 682, 1023] | 14 | [1, 2, 4, 8, 10] | 5 | True |
| 11 | [1, 2, 4, 8, 16, 32, 64, 128, 256, 288, 292, 584, 585, 1170, 1755, 2047] | 16 | [1, 2, 4, 8, 10, 11] | 6 | True |
| 12 | [1, 2, 4, 8, 16, 32, 64, 128, 256, 272, 273, 546, 1092, 1365, 2730, 4095] | 16 | [1, 2, 4, 8, 12] | 5 | True |


Исходя из полученных результатов можно судить о верности утверждения гипотезы Шольца-Брауэра для $ 1 \leq n \leq 12 $.

### 4. Оптимизация алгоритма дробления вектора индексов.

В ходе тестирования работы алгоритма дробления вектора индексов было замечено, что выбор индекса, разделяющего постоянную и меняющуюся части значительно влияет на время работы алгоритма.

Предложена идея варьировать размер постоянной части. Данную идею можно реализовать следующим образом: постепенно производить построение вектора индексов, на каждой итерации уже построенная часть вектора рассматривается как постоянная часть, перед продолжением построения будут вычисляться границы диапазона значений, которые могут быть получены при продолжении построения вектора ($a_{min}$ и $a_{max}$ в оригинальом алгоритме). Если искомое значение поподает в диапазон $[a_{min}, a_{max}]$, то происходит дальнейшее построение вектора, иначе происходит переход к следующей постоянной части.

В блоке кода, расположенном ниже, реализован предложенный алгоритм.

In [24]:
'''
    this function implements the optimization of index vector splitting algorithm
'''
def optimized_index_vector_splitting(n):
    if n == 1:
        return [1]
        
    vector_length = len(bin(n)) - 3
    
    # external loop on lengths of vectors
    while True:
    
        # index_stack has a similar significance as constant part of the vector of inexes 
        # from the unoptimized algorithm
        index_stack = [0 + 1]
        # in the process of the variation of the index vector the corresponding chain is constructed
        chain = [1, 2]
        
        # the cycle of sequential construction of the index vector
        while index_stack:
            # at the beginning of each iteration, the index at the top of the index stack decreases
            if not index_stack[-1]:
                # if this is not possible, the previous index variation begins
                index_stack.pop()
                chain.pop()
                continue
            
            # decreasing the index at the top of the stack and calculating the corresponding chain
            index_stack[-1] -= 1
            chain[-1] = chain[-2] + chain[index_stack[-1]]
            
            if chain[-1] == n:
                return chain
            
            # transition to the next chain if the current chain cannot be extended and is not a solution
            if len(index_stack) == vector_length:
                continue
            
            a_min = chain[-1] + vector_length - len(index_stack)
            a_max = chain[-1] * 2^(vector_length - len(index_stack))
            
            # transition to the next chain if the target value does not obtainable the range 
            # of possible values when expanding the current chain
            if not (a_min <= n <= a_max):
                continue
            
            # chain extension
            index_stack.append(len(index_stack) + 1)
            chain.append(None)
            
        vector_length += 1

Произведем вычисления, аналогичные тем, что были приведены в пункте 2.

In [25]:
def print_optimized_index_vector_splitting_info(n):
    start_time = time()
    chain = optimized_index_vector_splitting(n)
    print('n = {}, chain length = {}, time = {}\nchain = {}'.format(n, len(chain), time() - start_time, chain))

In [26]:
print_optimized_index_vector_splitting_info(1088)

n = 1088, chain length = 12, time = 0.00010514259338378906
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1088]


In [27]:
print_optimized_index_vector_splitting_info(1280)

n = 1280, chain length = 12, time = 7.987022399902344e-05
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1280]


In [28]:
print_optimized_index_vector_splitting_info(1160)

n = 1160, chain length = 13, time = 0.0020933151245117188
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1152, 1160]


In [29]:
print_optimized_index_vector_splitting_info(1172)

n = 1172, chain length = 13, time = 0.014542818069458008
chain = [1, 2, 4, 8, 16, 32, 64, 128, 132, 260, 520, 1040, 1172]


In [30]:
print_optimized_index_vector_splitting_info(1920)

n = 1920, chain length = 13, time = 0.0004172325134277344
chain = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 640, 1280, 1920]


Дополнительно произведем вычисления для $n = 2^9 - 1$.

In [31]:
print_index_vector_splitting_info(2^9-1)

n = 511, chain length = 13, time = 28.07638120651245
chain = [1, 2, 4, 8, 16, 32, 64, 72, 73, 146, 292, 438, 511]


In [32]:
print_optimized_index_vector_splitting_info(2^9-1)

n = 511, chain length = 13, time = 0.5105516910552979
chain = [1, 2, 4, 8, 16, 32, 64, 72, 73, 146, 292, 438, 511]


Полученные результаты сведены в таблицу, расположенную ниже.


| $n$ | Аддитивная цепочка | Время работы оптимизированного <br>алгоритма, с | Время работы неоптимизированного <br>алгоритма, с | Разность времени работы алгоритмов |
|---|---|---|---|---|
| 1088 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1088\] | 0.00014519691467285156 | 0.003418445587158203 | 0.0032732486724853516 |
| 1280 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1280\] | 5.9604644775390625e-05 | 0.006251096725463867 | 0.0061914920806884766 |
| 1160 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 1152, 1160\] | 0.0024764537811279297 | 0.4568898677825928 | 0.45441341400146484 |
| 1172 | \[1, 2, 4, 8, 16, 32, 64, 128, 132, 260, 520, 1040, 1172\] | 0.029340028762817383 | 0.5387921333312988 | 0.5094521045684814 |
| 1920 | \[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 640, 1280, 1920\] | 0.001153707504272461 | 0.05087733268737793 | 0.04972362518310547 |
| 511 | \[1, 2, 4, 8, 16, 32, 64, 72, 73, 146, 292, 438, 511\] | 0.49303507804870605 | 27.822351932525635 | 27.32931685447693 |

Оптимизированный алгоритм во всех рассмотренных случаях работает быстрее, чем исходный. Для $n = 511$ оптимизированный алгоритм находит решение в 56 раз быстрее, чем неоптимизированный.

## Выводы

В ходе выполнения работы был реализованы алгоритм Яо нахождения придлиженных аддитивных цепочек, алгоритм дробления вектора индексов для нахождения минимальной звездной аддитивной цепочки. Произведено сравнение данных алгоритмов, в результате которого получено, что алгоритм Яо не всегда может дать оптимальное решение, но при этом время работы алгоритма Яо значительно меньше, чем время работы алгоритма дробления вектора индексов. Для алгоритма дробления вектора индексов разработана оптимизация, основанная на варьировании длины постоянной части вектора индексов. Произведено сравнение оптимизированного и неоптимизированного алгоритмов дробления вектора индексов, в резуьтате которого показано, что оптимизированный алгоритм во всех рассмотренных случаях работает быстрее, чем неоптимизированный. 